In [1]:
import calliope
import pandas as pd
import plotly.express as px

calliope.set_log_verbosity("INFO", include_solver_output=False)

In [2]:
model = calliope.Model('model.yaml')

[2025-05-29 16:20:14] INFO     Model: initialising
[2025-05-29 16:20:14] INFO     Model: preprocessing stage 1 (model_run)
[2025-05-29 16:20:15] INFO     Model: preprocessing stage 2 (model_data)
[2025-05-29 16:20:15] INFO     Model: preprocessing complete


In [3]:
model.inputs

<xarray.Dataset> Size: 162kB
Dimensions:                 (costs: 1, techs: 6, nodes: 2, carriers: 2,
                             timesteps: 744)
Coordinates:
  * costs                   (costs) object 8B 'monetary'
  * techs                   (techs) object 48B 'aluminium_demand' ... 'fuel_s...
  * carriers                (carriers) object 16B 'aluminium' 'fuel'
  * nodes                   (nodes) <U11 88B 'Iceland' 'Netherlands'
  * timesteps               (timesteps) datetime64[ns] 6kB 2005-01-01 ... 200...
Data variables: (12/20)
    bigM                    float64 8B 1e+06
    objective_cost_weights  (costs) float64 8B 1.0
    base_tech               (techs) object 48B 'demand' 'supply' ... 'supply'
    carrier                 (nodes, techs) object 96B nan 'aluminium' ... nan
    carrier_out             (nodes, techs, carriers) float64 192B nan ... nan
    color                   (techs) object 48B '#072486' '#F9CF22' ... '#E37A72'
    ...                      ...
    sink_use_max            (timesteps, nodes, techs) float64 71kB nan ... nan
    source_use_max          (timesteps, nodes, techs) float64 71kB nan ... nan
    definition_matrix       (nodes, techs, carriers) bool 24B False ... False
    distance                (techs) float64 48B nan nan 2.021e+03 ... nan nan
    timestep_resolution     (timesteps) float64 6kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
    timestep_weights        (timesteps) float64 6kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
Attributes:
    calliope_version_defined:      0.7.0
    calliope_version_initialised:  0.7.0.dev6
    applied_overrides:             
    scenario:                      None
    defaults:                      {'bigM': 1000000000.0, 'objective_cost_wei...
    allow_operate_mode:            1
    name:                          main model
    timestamp_model_creation:      1748528414.670124

In [4]:
model.inputs.flow_cap_max.to_series().dropna()

techs
aluminium_supply               inf
aluminium_transport_tech    3000.0
fuel_consumption_tech          7.0
fuel_supply                    inf
Name: flow_cap_max, dtype: float64

In [5]:
model.build(force=True)
model.solve()

[2025-05-29 16:20:15] INFO     Model: backend build starting
[2025-05-29 16:20:15] INFO     Math preprocessing | added file 'plan'.
[2025-05-29 16:20:15] INFO     Math preprocessing | added file 'custom_constraints_state.yml'.
[2025-05-29 16:20:15] INFO     Math preprocessing | added file 'custom_constraints_delay.yml'.
[2025-05-29 16:20:15] INFO     Math preprocessing | validated math against schema.
[2025-05-29 16:20:15] INFO     Optimisation Model | parameters | Generated.
[2025-05-29 16:20:17] INFO     Optimisation Model | Validated math strings.
[2025-05-29 16:20:17] INFO     Optimisation Model | variables | Generated.
[2025-05-29 16:20:18] INFO     Optimisation Model | global_expressions | Generated.
[2025-05-29 16:20:19] INFO     Optimisation Model | constraints | Generated.
[2025-05-29 16:20:19] INFO     Optimisation Model | piecewise_constraints | Generated.
[2025-05-29 16:20:19] INFO     Optimisation Model | objectives | Generated.
[2025-05-29 16:20:19] INFO     Model: backen

In [6]:
model.results

<xarray.Dataset> Size: 935kB
Dimensions:                     (nodes: 2, techs: 6, carriers: 2,
                                 timesteps: 744, costs: 1)
Coordinates:
  * techs                       (techs) object 48B 'aluminium_demand' ... 'fu...
  * nodes                       (nodes) <U11 88B 'Iceland' 'Netherlands'
  * carriers                    (carriers) <U9 72B 'aluminium' 'fuel'
  * timesteps                   (timesteps) datetime64[ns] 6kB 2005-01-01 ......
  * costs                       (costs) object 8B 'monetary'
Data variables: (12/16)
    flow_cap                    (nodes, techs, carriers) float64 192B nan ......
    link_flow_cap               (techs) float64 48B nan nan 3e+03 7.0 nan nan
    flow_out                    (nodes, techs, carriers, timesteps) float64 143kB ...
    flow_in                     (nodes, techs, carriers, timesteps) float64 143kB ...
    source_use                  (nodes, techs, timesteps) float64 71kB nan .....
    source_cap                  (nodes, techs) float64 96B nan 3e+03 ... nan nan
    ...                          ...
    cost_operation_variable     (nodes, techs, costs, timesteps) float64 71kB ...
    cost                        (nodes, techs, costs) float64 96B nan ... nan
    capacity_factor             (nodes, techs, carriers, timesteps) float64 143kB ...
    systemwide_capacity_factor  (techs, carriers) float64 96B 0.0 0.0 ... 1.0
    systemwide_levelised_cost   (carriers, techs, costs) float64 96B nan ... ...
    total_levelised_cost        (carriers, costs) float64 16B -1.345 -3.872
Attributes: (12/13)
    termination_condition:         optimal
    calliope_version_defined:      0.7.0
    calliope_version_initialised:  0.7.0.dev6
    applied_overrides:             
    scenario:                      None
    defaults:                      {'bigM': 1000000000.0, 'objective_cost_wei...
    ...                            ...
    name:                          main model
    timestamp_model_creation:      1748528414.670124
    timestamp_build_start:         1748528415.410526
    timestamp_build_complete:      1748528419.515047
    timestamp_solve_start:         1748528419.516046
    timestamp_solve_complete:      1748528420.287891

In [7]:
costs = model.results.cost.to_series().dropna()
costs.head()

nodes        techs             costs   
Iceland      aluminium_supply  monetary       37.500
             fuel_supply       monetary        2.604
Netherlands  aluminium_demand  monetary   -15000.000
             fuel_necessity    monetary    -5208.000
Name: cost, dtype: float64

In [8]:
lcoes = model.results.systemwide_levelised_cost.to_series().dropna()
lcoes

carriers   techs             costs   
aluminium  aluminium_supply  monetary    0.0025
fuel       fuel_supply       monetary    0.0005
Name: systemwide_levelised_cost, dtype: float64

In [9]:
flow_out = model.results.flow_out.sel(carriers="aluminium").where(lambda x: x != 0).to_series().dropna()
flow_out

nodes        techs                     timesteps          
Iceland      aluminium_supply          2005-01-01 00:00:00    3000.0
                                       2005-01-07 02:00:00    3000.0
                                       2005-01-13 04:00:00    3000.0
                                       2005-01-19 05:00:00    3000.0
                                       2005-01-25 08:00:00    3000.0
Netherlands  aluminium_transport_tech  2005-01-04 00:00:00    3000.0
                                       2005-01-10 02:00:00    3000.0
                                       2005-01-16 04:00:00    3000.0
                                       2005-01-22 05:00:00    3000.0
                                       2005-01-28 08:00:00    3000.0
Name: flow_out, dtype: float64

In [10]:
colors = model.inputs.color.to_series().to_dict()

In [11]:
model.to_netcdf("simple_model.nc")